In [ ]:
import json
import hmac
import time
import base64
import urllib3
import hashlib
import requests
import pandas as pd
from requests.auth import AuthBase

In [ ]:
api_cred = pd.read_json("../API_KEYS/api_keys.json", orient= "index")
api_url = pd.read_json("../API_KEYS/api_url.json", orient= "index")

In [ ]:
api_url_gdax = api_url.loc["GDAX"].URL

In [ ]:
# Create custom authentication for Exchange
class GDAXExchangeAuth(AuthBase):
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET
        self.passphrase = api_cred.loc[exchange].PASSPHRASE

    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [ ]:
auth = GDAXExchangeAuth(exchange="GDAX")
auth

In [ ]:
# Get accounts
balance_data = requests.get(api_url_gdax + 'accounts', auth=auth)
balance_data = balance_data.json()
balance_data = pd.DataFrame(balance_data)
balance_data.drop("profile_id", axis=1, inplace=True)
balance_data["product_id"] = balance_data.currency+"-USD"
balance_data.balance = pd.to_numeric(balance_data.balance)
balance_data

In [ ]:
type(balance_data.balance)

In [ ]:
balance_data.balance.dtype

In [ ]:
account_history = []

for acct in balance_data.id:
    account = 'accounts/'+ acct +'/ledger'
    acct_hist = requests.get(api_url_gdax + account, auth=auth)
    temp = acct_hist.json()
    temp = pd.DataFrame(temp)
    # to split the dict at "details" and convert it to columns
    temp = pd.concat([temp.drop(['details'], axis=1), temp['details'].apply(pd.Series)], axis=1) 
    account_history.append(temp)

account_history = pd.concat(account_history, axis=0)
account_history = account_history.reset_index()
account_history.drop("index", 1, inplace=True)
account_history

In [ ]:
msk = account_history.product_id.isnull()
product_history = account_history[~msk]
tranfer_history = account_history[msk]

In [ ]:
tranfer_history

In [ ]:
# get latest price for a product
product_price = []
for prdct in product_history.product_id.unique():
    url = api_url_gdax+"products/"+ prdct + "/ticker"
    ticker_info = requests.get(url, auth = auth)
    ticker = ticker_info.json()
    ticker = pd.DataFrame(ticker, index=[0])
    ticker["product_id"] = prdct
#     print(prdct, ticker)
    product_price.append(ticker)
    
product_price = pd.concat(product_price)
product_price = product_price.reset_index()

In [ ]:
product_price

In [ ]:
portfolio = balance_data.join(product_price.set_index('product_id'), on = 'product_id', how="left")
portfolio.bid = pd.to_numeric(portfolio.bid)

In [ ]:
portfolio.bid.dtype

In [ ]:
portfolio.balance.dtype

In [ ]:
portfolio["curr_val"] = portfolio.balance * portfolio.bid
portfolio.curr_val.sum()